WARNING: old version, now runs automatically as onroerend_erfgoed_auto.py via cron!
-
-
-
-
-
-
-

In [8]:
# install libraries 
!pip install pyproj
!pip install -U numpy==1.26.4
!pip install transformers
!pip install -U accelerate
!pip install pandas
!pip install PyPDF2
!pip install NLTK
!pip install editdistance
!pip install  paramiko
!pip install func_timeout

# import nltk / download punkt
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /home/alex/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/alex/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
# import other libraries
import urllib.request
import requests
import json
from pyproj import Transformer
import os
import time
import sys
import nltk

# import common functions
sys.path.insert(1, '../')
import common

# set up transformer from Lambert72 to lat/lon
transformer = Transformer.from_crs("EPSG:31370", "EPSG:4326")

# set name of module, to fetch info from config
module_name = "onroerend_erfgoed"

In [6]:
# get info from config file
config = common.get_config()
        
last_indexed_id = config['data_source'][module_name]['last_indexed_id']
print(f'last indexed id: {last_indexed_id}')

endpoint_url = config['data_source'][module_name]['endpoint_url']
print(f'endpoint_url: {endpoint_url}')

pdf_folder = config['data_source'][module_name]['pdf_folder']
print(f'pdf_folder: {pdf_folder}')

json_folder = config['data_source'][module_name]['json_folder']
print(f'json_folder: {json_folder}')

html_folder = config['data_source'][module_name]['html_folder']
print(f'html_folder: {html_folder}')

language = config['data_source'][module_name]['language']
print(f'language: {language}')

bert_model = config['bert_models'][language]
print(f'bert_model: {bert_model}')

last indexed id: 3039
endpoint_url: https://loket.onroerenderfgoed.be/archeologie/rapporten/eindverslagen/
pdf_folder: /media/alex/Data/agnes_data/onroerend_erfgoed/pdf/
json_folder: /media/alex/Data/agnes_data/onroerend_erfgoed/json/
html_folder: /media/alex/Data/agnes_data/onroerend_erfgoed/html/
language: dutch
bert_model: /media/alex/Data/agnes_models/ArcheoBERTje-NER


In [3]:
# set headers
headers = {'Accept': 'application/json', 
           'User-Agent': 'python-requests/2.18.1',
           'From': 'a.brandsen@arch.leidenuniv.nl (EXALT)'  
          }

# set parameters
# https://loket.onroerenderfgoed.be/archeologie/rapporten/eindverslagen?sort=id&per_pagina=50&pagina=1
params = {'per_pagina': 100, # number of results per page
          'sort': 'id',
          #'pagina': 1 # page 
         }   


In [4]:
# Create a requests Session
session = requests.Session()

# Set the header options
session.headers.update(headers)

# Make a request and store the response
try:
    response = session.get(endpoint_url, params=params)
    response.raise_for_status()                 # Raise error in case of failure 
except requests.exceptions.HTTPError as httpErr: 
    print ("HTTP Error:",httpErr) 
except requests.exceptions.ConnectionError as connErr: 
    print ("Error Connecting:",connErr) 
except requests.exceptions.Timeout as timeOutErr: 
    print ("Timeout Error:",timeOutErr) 
except requests.exceptions.RequestException as reqErr: 
    print ("Something Else:",reqErr)
    
# get json data from request
data = response.json()

# Fetch all the other pages of results
while 'next' in response.links:
    # GET the url pointing to the next page
    response = session.get(response.links['next']['url'])
    # add the data from the next page to the data we're collecting
    data.extend(response.json())
    # Sleep for half a second, so we don't overload the server
    time.sleep(0.5)
  
print(len(data))

1339


In [7]:
# go through projects
for project in data:
    
    #print(json.dumps(project, indent=4))

    # if not fetched yet
    if project['id'] > last_indexed_id:

        print(f"Processing id:{project['id']}")
    
        # get more details on project using uri
        response = requests.get(
            project['uri'],
            headers = {'Accept': 'application/json'}
        )
        project_detail = response.json()

        #print(json.dumps(project_detail, sort_keys=True, indent=4))

        # loop through relevant documents, make document.json for each
        for doc in project_detail['bijlagen']:

            #print(json.dumps(doc, sort_keys=True, indent=4))
            
            if doc['bijlagetype']['id'] == 2 and doc['bestandsnaam'][-3:].lower() == 'pdf': # is report and is pdf file
                #print(project)

                file_name = doc['bestandsnaam'].replace(' ','_').replace('(','').replace(')','')

                output_document = {}
                output_document['source'] = module_name
                output_document['file_name'] = file_name
                output_document['file_type'] = 'report'
                if(doc['omschrijving']):
                    title = f"{project['onderwerp']} - {doc['omschrijving']}"
                else:
                    title = f"{project['onderwerp']}"
                output_document['title'] = title
                creators = []
                for auteur in doc['auteurs']:
                    creators.append(auteur['auteur'])
                output_document['creators'] = creators
                output_document['description'] = '' # no descriptions in this data source
                output_document['publisher'] = project['archeoloog']
                output_document['createdAt'] = project['datum_indienen'][:10]
                output_document['identifiers'] = {'uri':project['uri'],'oe_id':project['id']}
                output_document['language'] = 'Dutch'

                #coordinates
                totalX = 0
                totalY = 0
                coordinates = project_detail['archeologierapport']['locatie']['contour']['coordinates'][0][0]
                for coord in coordinates:
                    totalX += coord[0]
                    totalY += coord[1]
                coordX = totalX / len(coordinates)
                coordY = totalY / len(coordinates)
                lat, lon = transformer.transform(coordX,coordY)
                output_document['coordX'] = coordX
                output_document['coordY'] = coordY
                output_document['location'] = {'lat':lat,'lon':lon}

                #print(json.dumps(output_document, indent=4))

                # set document identifier
                doc_id = f"{project['id']}_{file_name.replace('.pdf','')}"

                print(f"doc_id:{doc_id}")

                # save document.json 
                json_output_folder = f"{json_folder}/{doc_id}"
                common.savejson(output_document, f"{json_output_folder}/document.json")

                print(f"saved doc json")

                # download pdf
                file_url = doc['url']
                pdf_location = common.downloaddocument(file_url, project['id'], pdf_folder, file_name)

                print(f"downloaded pdf")

                # process pdf, store page.json files with entities 
                common.run_ner_on_pdf(
                    pdf_location, 
                    json_output_folder, 
                    bert_model, 
                    language
                )

                print(f"ran NER, saved page json")

                # process pdf, save html files
                html_output_folder = f"{html_folder}/{doc_id}"
                common.pdf2html(pdf_location, html_output_folder)

                print(f"generated and saved html")

                # save last id we indexed in the settings file 
                # (do this in the loop instead of after, in case we get errors/hanging that ends the script before we get to the end)
                common.update_config(module_name, 'last_indexed_id', project['id'])
                
                print(f"updated config with latest id: {project['id']}")

                
# upload json and html to webserver
common.upload2webserver(json_folder, html_folder, module_name, config['webserver']['json_folder'], config['webserver']['html_folder'])

print(f"uploaded json/html to webserver")

# remotely start indexing script on webserver
common.start_index(module_name)

print(f"indexing on webserver started")

print(f"done!")



Processing id:3048
doc_id:3048_HEKE03_2024G23_Opgraving_Eindverslag
saved doc json


Superfluous whitespace found in object header b'1' b'0'
Superfluous whitespace found in object header b'2' b'0'
Superfluous whitespace found in object header b'3' b'0'
Superfluous whitespace found in object header b'36' b'0'
Superfluous whitespace found in object header b'79' b'0'
Superfluous whitespace found in object header b'82' b'0'
Superfluous whitespace found in object header b'85' b'0'
Superfluous whitespace found in object header b'88' b'0'
Superfluous whitespace found in object header b'99' b'0'
Superfluous whitespace found in object header b'104' b'0'
Superfluous whitespace found in object header b'115' b'0'
Superfluous whitespace found in object header b'119' b'0'
Superfluous whitespace found in object header b'122' b'0'
Superfluous whitespace found in object header b'126' b'0'
Superfluous whitespace found in object header b'132' b'0'
Superfluous whitespace found in object header b'143' b'0'
Superfluous whitespace found in object header b'146' b'0'
Superfluous whitespace fou

downloaded pdf


Superfluous whitespace found in object header b'78' b'0'
Superfluous whitespace found in object header b'44' b'0'
Superfluous whitespace found in object header b'43' b'0'
Superfluous whitespace found in object header b'42' b'0'
Superfluous whitespace found in object header b'52' b'0'
Superfluous whitespace found in object header b'51' b'0'
Superfluous whitespace found in object header b'50' b'0'
Superfluous whitespace found in object header b'60' b'0'
Superfluous whitespace found in object header b'59' b'0'
Superfluous whitespace found in object header b'58' b'0'
Superfluous whitespace found in object header b'68' b'0'
Superfluous whitespace found in object header b'67' b'0'
Superfluous whitespace found in object header b'66' b'0'
Superfluous whitespace found in object header b'76' b'0'
Superfluous whitespace found in object header b'75' b'0'
Superfluous whitespace found in object header b'74' b'0'
Superfluous whitespace found in object header b'77' b'0'
Superfluous whitespace found in

ran NER, saved page json
Converted /media/alex/Data/agnes_data/onroerend_erfgoed/pdf/3048_HEKE03_2024G23_Opgraving_Eindverslag.pdf to html!
generated and saved html
updated config with latest id: 3048
Processing id:3049
doc_id:3049_Eindverslag_Bonheiden_-_Hoogstraat_v001
saved doc json
downloaded pdf
ran NER, saved page json
Converted /media/alex/Data/agnes_data/onroerend_erfgoed/pdf/3049_Eindverslag_Bonheiden_-_Hoogstraat_v001.pdf to html!
generated and saved html
updated config with latest id: 3049
Processing id:3050
doc_id:3050_2018-0530_Gent_Hoogstraat_EV_DEF
saved doc json
downloaded pdf
ran NER, saved page json
Converted /media/alex/Data/agnes_data/onroerend_erfgoed/pdf/3050_2018-0530_Gent_Hoogstraat_EV_DEF.pdf to html!
generated and saved html
updated config with latest id: 3050
Processing id:3051
doc_id:3051_TR2021-018_Rumst_Kasteel_Eindverslag_V100
saved doc json
downloaded pdf
ran NER, saved page json
Converted /media/alex/Data/agnes_data/onroerend_erfgoed/pdf/3051_TR2021-018

...

downloaded pdf


Superfluous whitespace found in object header b'73' b'0'
Superfluous whitespace found in object header b'62' b'0'
Superfluous whitespace found in object header b'61' b'0'
Superfluous whitespace found in object header b'60' b'0'
Superfluous whitespace found in object header b'71' b'0'
Superfluous whitespace found in object header b'70' b'0'
Superfluous whitespace found in object header b'69' b'0'
Superfluous whitespace found in object header b'72' b'0'
Superfluous whitespace found in object header b'63' b'0'
Superfluous whitespace found in object header b'78' b'0'
Superfluous whitespace found in object header b'77' b'0'
Superfluous whitespace found in object header b'75' b'0'
Superfluous whitespace found in object header b'76' b'0'
Superfluous whitespace found in object header b'82' b'0'
Superfluous whitespace found in object header b'81' b'0'
Superfluous whitespace found in object header b'80' b'0'
Superfluous whitespace found in object header b'149' b'0'
Superfluous whitespace found i

ran NER, saved page json
Converted /media/alex/Data/agnes_data/onroerend_erfgoed/pdf/3207_2022-0373_Hamont-Achel_Klein_Bien_EV.pdf to html!
generated and saved html
updated config with latest id: 3207
Processing id:3210
doc_id:3210_Bijlage_1
saved doc json
downloaded pdf
ran NER, saved page json
Converted /media/alex/Data/agnes_data/onroerend_erfgoed/pdf/3210_Bijlage_1.pdf to html!
generated and saved html
updated config with latest id: 3210
doc_id:3210_Eindrapport-Sint-Lievens-Houtem_Eilandbos_DIG
saved doc json
downloaded pdf
ran NER, saved page json
Converted /media/alex/Data/agnes_data/onroerend_erfgoed/pdf/3210_Eindrapport-Sint-Lievens-Houtem_Eilandbos_DIG.pdf to html!
generated and saved html
updated config with latest id: 3210
doc_id:3210_Bijlage_3
saved doc json
downloaded pdf
ran NER, saved page json
Converted /media/alex/Data/agnes_data/onroerend_erfgoed/pdf/3210_Bijlage_3.pdf to html!
generated and saved html
updated config with latest id: 3210
doc_id:3210_Bijlage_2
saved do

Xref table not zero-indexed. ID numbers for objects will be corrected.


downloaded pdf
ran NER, saved page json
Converted /media/alex/Data/agnes_data/onroerend_erfgoed/pdf/3233_Eindverslag_opgravingen_Heilige_Geeststraat-lowres.pdf to html!
generated and saved html
updated config with latest id: 3233
Processing id:3235
doc_id:3235_2020-0708_Bonheiden_Rijmenam_Pikhaken_EV
saved doc json
downloaded pdf
ran NER, saved page json
Converted /media/alex/Data/agnes_data/onroerend_erfgoed/pdf/3235_2020-0708_Bonheiden_Rijmenam_Pikhaken_EV.pdf to html!
generated and saved html
updated config with latest id: 3235
Processing id:3244
doc_id:3244_OAOE349-001
saved doc json
downloaded pdf
ran NER, saved page json
Converted /media/alex/Data/agnes_data/onroerend_erfgoed/pdf/3244_OAOE349-001.pdf to html!
generated and saved html
updated config with latest id: 3244
Processing id:3249
doc_id:3249_2023-089_Asse_Fietssnelweg_F212-F221_Eindverslag
saved doc json
downloaded pdf
ran NER, saved page json
Converted /media/alex/Data/agnes_data/onroerend_erfgoed/pdf/3249_2023-089_Asse_

In [ ]:
# get more details on project using uri
response = requests.get(
    'https://loket.onroerenderfgoed.be/archeologie/notas/notas/44',
    headers = {'Accept': 'application/json'}
)
project_detail = response.json()

In [ ]:
print(project_detail)